##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Validação de dados usando TFX Pipeline e o TensorFlow Data Validation

Observação: recomendamos executar este tutorial em um notebook Colab, sem necessidade de configuração! Basta clicar em “Executar no Google Colab”.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

Neste tutorial baseado em notebook, criaremos e executaremos pipelines TFX para validar dados de entrada e criar um modelo de ML. Este notebook é baseado no pipeline TFX que construímos no [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple). Se você ainda não leu esse tutorial, leia-o antes de prosseguir com este notebook.

A primeira tarefa em qualquer projeto de ciência de dados ou ML é compreender e limpar os dados, o que inclui:

- Compreensão dos tipos de dados, distribuições e outras informações (por exemplo, valor médio ou número de valores únicos) sobre cada característica
- Geração de um esquema preliminar que descreva os dados
- Identificação de anomalias e valores ausentes nos dados em relação a determinado esquema

Neste tutorial, criaremos dois pipelines TFX.

Primeiro, criaremos um pipeline para analisar o dataset e gerar um esquema preliminar do dataset fornecido. Este pipeline incluirá dois novos componentes, `StatisticsGen` e `SchemaGen`.

Assim que tivermos um esquema adequado dos dados, criaremos um pipeline para treinar um modelo de classificação de ML com base no pipeline do tutorial anterior. Neste pipeline, usaremos o esquema do primeiro pipeline e um novo componente, `ExampleValidator`, para validar os dados de entrada.

Os três novos componentes, StatisticsGen, SchemaGen e ExampleValidator, são componentes TFX para análise e validação de dados e são implementados usando a biblioteca [TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv).

Veja [Introdução aos pipelines do TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para saber mais sobre vários conceitos do TFX.

## Configuração

Primeiro precisamos instalar o pacote TFX Python e baixar o dataset que usaremos para nosso modelo.

### Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instale o TFX


In [ ]:
!pip install -U tfx

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

### Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime clicando no botão "RESTART RUNTIME" acima ou usando o menu "Runtime &gt; Restart runtime ...". Isso é necessário devido à maneira como o Colab carrega os pacotes.

Verifique as versões do TensorFlow e TFX.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### Configuração de variáveis

Existem algumas variáveis ​​usadas para definir um pipeline. Você pode personalizar essas variáveis ​​como desejar. Por padrão, toda a saída do pipeline será gerada no diretório atual.

In [ ]:
import os

# We will create two pipelines. One for schema generation and one for training.
SCHEMA_PIPELINE_NAME = "penguin-tfdv-schema"
PIPELINE_NAME = "penguin-tfdv"

# Output directory to store artifacts generated from the pipeline.
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Preparação dos dados de exemplo

Faremos download do dataset de exemplo para uso no nosso pipeline TFX. O dataset que estamos usando é o [dataset Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) , que também é usado em outros [exemplos com TFX](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin).

Há quatro características numéricas neste dataset:

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

Todas as características já foram normalizadas para ter um intervalo [0,1]. Construiremos um modelo de classificação que prevê as `species` de pinguins.

Como o componente TFX ExampleGen lê entradas de um diretório, precisamos criar um diretório e copiar o dataset para ele.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

Dê uma olhada rápida no arquivo CSV.

In [ ]:
!head {_data_filepath}

Você deve conseguir ver cinco colunas de características. `species` é 0, 1 ou 2, e todas as outras características devem ter valores entre 0 e 1. Criaremos um pipeline TFX para analisar este dataset.

## Gere um esquema preliminar

Os pipelines TFX são definidos usando APIs Python. Criaremos um pipeline para gerar automaticamente um esquema a partir dos exemplos de entrada. Este esquema pode ser revisado por um humano e ajustado conforme necessário. Depois que o esquema for finalizado, ele poderá ser usado para treinamento e validação de exemplos em tarefas posteriores.

Além de `CsvExampleGen` que é usado no [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple), usaremos `StatisticsGen` e `SchemaGen`:

- [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) calcula estatísticas para o dataset.
- [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) examina as estatísticas e cria um esquema de dados inicial.

Consulte os guias de cada componente ou [o tutorial dos componentes TFX](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) para saber mais sobre esses componentes.

### Escreva uma definição de pipeline

Definimos uma função para criar um pipeline TFX. Um objeto `Pipeline` representa um pipeline do TFX que pode ser executado usando um dos sistemas de orquestração de pipeline suportados pelo TFX.

In [ ]:
def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Generates schema based on the generated statistics.
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### Execute o pipeline

Usaremos o `LocalDagRunner` como no tutorial anterior.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=SCHEMA_METADATA_PATH))

Você deverá ver "INFO:absl:Component SchemaGen is finished." se o pipeline terminou com sucesso.

Examinaremos a saída do pipeline para entender nosso dataset.

### Examine as saídas do pipeline

Conforme explicado no tutorial anterior, um pipeline TFX produz dois tipos de saídas, artefatos e um [banco de dados de metadados (MLMD)](https://www.tensorflow.org/tfx/guide/mlmd) que contém metadados de artefatos e execuções de pipeline. Definimos a localização dessas saídas nas células acima. Por padrão, os artefatos são armazenados no diretório `pipelines` e os metadados são armazenados como um banco de dados sqlite no diretório `metadata`.

Você pode usar APIs MLMD para localizar essas saídas programaticamente. Primeiro, definiremos algumas funções utilitárias para procurar os artefatos de saída que acabaram de ser produzidos.


In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

Agora podemos examinar as saídas da execução do pipeline.

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

É hora de examinar os resultados de cada componente. Conforme descrito acima, [Tensorflow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) é usado em `StatisticsGen` e `SchemaGen`, e o TFDV também fornece visualização das saídas desses componentes.

Neste tutorial, usaremos os métodos helper de visualização no TFX que usam TFDV internamente para mostrar a visualização.

#### Examine a saída do StatisticsGen


In [ ]:
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

<!-- <img class="tfo-display-only-on-site"
src="images/penguin_tfdv/penguin_tfdv_statistics.png"/> -->

Você pode ver várias estatísticas para os dados de entrada. Essas estatísticas são fornecidas ao `SchemaGen` para construir automaticamente um esquema inicial de dados.


#### Examine a saída do SchemaGen


In [ ]:
visualize_artifacts(schema_artifacts)

Este esquema é inferido automaticamente da saída do StatisticsGen. Você deverá conseguir ver 4 recursos FLOAT e 1 recurso INT.

### Exporte o esquema para uso futuro

Precisamos revisar e refinar o esquema gerado. O esquema revisado precisa ser persistido para ser usado em pipelines subsequentes para treinamento do modelo de ML. Em outras palavras, você pode querer adicionar o arquivo de esquema ao seu sistema de controle de versão para casos de uso reais. Neste tutorial, iremos apenas copiar o esquema para um caminho de sistema de arquivos predefinido para simplificar.


In [ ]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

O arquivo de esquema usa o [formato de texto Protocol Buffer](https://googleapis.dev/python/protobuf/latest/google/protobuf/text_format.html) e uma instância do [TensorFlow Metadata Schema proto](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto).

In [ ]:
print(f'Schema at {SCHEMA_PATH}-----')
!cat {SCHEMA_PATH}/*

Você deve revisar e possivelmente editar a definição do esquema conforme necessário. Neste tutorial, usaremos apenas o esquema gerado inalterado.


## Valide exemplos de entrada e treine um modelo de ML

Voltaremos ao pipeline que criamos no [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple), para treinar um modelo de ML e usar o esquema gerado para escrever o código de treinamento do modelo.

Também adicionaremos um componente [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) que procurará anomalias e valores ausentes no dataset recebido em relação ao esquema.


### Escreva o código para treinamento do modelo

Precisamos escrever o código do modelo como fizemos no [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

O modelo em si é o mesmo do tutorial anterior, mas desta vez usaremos o esquema gerado no pipeline anterior em vez de especificar os recursos manualmente. A maior parte do código não foi alterada. A única diferença é que não precisamos especificar os nomes e tipos de recursos neste arquivo. Em vez disso, nós os lemos no arquivo de *esquema*.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

# We don't need to specify _FEATURE_KEYS and _FEATURE_SPEC any more.
# Those information can be read from the given schema file.

_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model(schema: schema_pb2.Schema) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.

  # ++ Changed code: Uses all features in the schema except the label.
  feature_keys = [f.name for f in schema.feature if f.name != _LABEL_KEY]
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in feature_keys]
  # ++ End of the changed code.

  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # ++ Changed code: Reads in schema file passed to the Trainer component.
  schema = tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema_pb2.Schema())
  # ++ End of the changed code.

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model(schema)
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Agora você concluiu todas as etapas de preparação para criar um pipeline do TFX para treinamento de modelos.

### Escreva uma definição de pipeline

Adicionaremos dois novos componentes, `Importer` e `ExampleValidator`. O Importer traz um arquivo externo para o pipeline do TFX. Neste caso, é um arquivo contendo a definição do esquema. ExampleValidator examinará os dados de entrada e validará se todos os dados de entrada estão em conformidade com o esquema de dados que fornecemos.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline using predefined schema with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # NEW: Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],  # Pass the imported schema.
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,

      # NEW: Following three components were added to the pipeline.
      statistics_gen,
      schema_importer,
      example_validator,

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### Execute o pipeline


In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

Você deverá ver "INFO:absl:Component Pusher is finished." se o pipeline for concluído com sucesso.

### Examine as saídas do pipeline

Treinamos o modelo de classificação para pinguins e também validamos os exemplos de entrada no componente ExampleValidator. Podemos analisar a saída de ExampleValidator como fizemos com o pipeline anterior.

In [ ]:
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  ev_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                   'ExampleValidator')
  anomalies_artifacts = ev_output[standard_component_specs.ANOMALIES_KEY]

Anomalias do ExampleValidator também podem ser visualizadas.

In [ ]:
visualize_artifacts(anomalies_artifacts)

Você deverá ver "No anomalies found" para cada divisão de exemplos. Como usamos os mesmos dados que foram usados ​​para a geração do esquema neste pipeline, nenhuma anomalia é esperada aqui. Se você executar esse pipeline repetidamente com novos dados recebidos, o ExampleValidator deverá ser capaz de encontrar quaisquer discrepâncias entre os novos dados e o esquema existente.

Se alguma anomalia for encontrada, você poderá revisar seus dados para verificar se algum exemplo não segue suas suposições. Os resultados de outros componentes como StatisticsGen podem ser úteis. No entanto, quaisquer anomalias encontradas NÃO bloquearão outras execuções do pipeline.

## Próximos passos

Você encontrará mais recursos em https://www.tensorflow.org/tfx/tutorials.

Veja [Introdução aos pipelines do TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para saber mais sobre vários conceitos do TFX.
